In [2]:
pip install xgboost


   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB 5.2 MB/s eta 0:00:24
   ---------------------------------------- 0.3/124.9 MB 5.3 MB/s eta 0:00:24
   ---------------------------------------- 0.3/124.9 MB 5.3 MB/s eta 0:00:24
   ---------------------------------------- 0.5/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 0.5/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 0.5/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 0.5/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 0.5/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 0.5/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 0.5/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 0.5/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 0.6/124.9 MB 1.0 MB/s eta 

In [ ]:
#more modeling for outage duration predictions
#Business Use Case:
#If predicting outage duration for maintenance or emergency preparedness, 
#A smaller RMSE can save costs or reduce downtime, making even small improvements valuable.

In [8]:
#imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor

outage_df=pd.read_csv('Trainingdata.csv')

In [6]:
outage_df.head()

,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags,Event Start,Event End,Duration,Duration (Hours),Description Length,Duration (Hours)_scaled
0,Severe Weather - Thunderstorms,2014,2014-06-30,20:00:00,2014-07-02,18:30:00,Exelon Corporation/ComEd,Illinois,RFC,Unknown,"420,000","severe weather, thunderstorm",2014-06-30 20:00:00,2014-07-02 18:30:00,1 days 22:30:00,46.500000,30,-0.003539
1,Severe Weather - Thunderstorms,2014,2014-06-30,23:20:00,2014-07-01,17:00:00,Northern Indiana Public Service Company,North Central Indiana,RFC,Unknown,"127,000","severe weather, thunderstorm",2014-06-30 23:20:00,2014-07-01 17:00:00,0 days 17:40:00,17.666667,30,-0.235446
2,Severe Weather - Thunderstorms,2014,2014-06-30,17:55:00,2014-07-01,02:53:00,We Energies,Southeast Wisconsin,MRO,424,"120,000","severe weather, thunderstorm",2014-06-30 17:55:00,2014-07-01 02:53:00,0 days 08:58:00,8.966667,30,-0.305420
3,Physical Attack - Vandalism,2014,2014-06-24,14:54:00,2014-06-24,14:55:00,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical",2014-06-24 14:54:00,2014-06-24 14:55:00,0 days 00:01:00,0.016667,27,-0.377405
4,Physical Attack - Vandalism,2014,2014-06-19,08:47:00,2014-06-19,08:48:00,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical",2014-06-19 08:47:00,2014-06-19 08:48:00,0 days 00:01:00,0.016667,27,-0.377405


In [10]:
# Option 2: Impute missing values (e.g., with median)
outage_df['Duration (Hours)'].fillna(outage_df['Duration (Hours)'].median(), inplace=True)


In [11]:
# Encode categorical variables
label_encoder = LabelEncoder()
outage_df['Event Description'] = label_encoder.fit_transform(outage_df['Event Description'])
outage_df['Respondent'] = label_encoder.fit_transform(outage_df['Respondent'])

# Define features (X) and target (y)
X = outage_df[['Event Description', 'Year', 'Respondent']]
y = outage_df['Duration (Hours)']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = linear_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 112.6998886602077


In [12]:
# Initialize and train the model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = linear_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Linear Regression RMSE: {rmse}")

Linear Regression RMSE: 112.6998886602077


In [13]:
# Initialize and train the model
gbr = GradientBoostingRegressor(n_estimators=100, random_state=42)
gbr.fit(X_train, y_train)

# Predict and evaluate
y_pred = gbr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Gradient Boosting RMSE: {rmse}")

Gradient Boosting RMSE: 160.58032345550419


In [14]:
# Initialize and train the model
svr = SVR(kernel='rbf')
svr.fit(X_train, y_train)

# Predict and evaluate
y_pred = svr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Support Vector Regressor RMSE: {rmse}")

Support Vector Regressor RMSE: 116.86468317623566


In [15]:
# Initialize and train the model
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"XGBoost RMSE: {rmse}")

XGBoost RMSE: 135.2227165532558


In [16]:
# Initialize and train the model
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

# Predict and evaluate
y_pred = knn.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"K-Nearest Neighbors RMSE: {rmse}")

K-Nearest Neighbors RMSE: 126.51683842710449


In [17]:
# Initialize and train the model
ann = MLPRegressor(hidden_layer_sizes=(100,100), max_iter=500, random_state=42)
ann.fit(X_train, y_train)

# Predict and evaluate
y_pred = ann.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Neural Network RMSE: {rmse}")

Neural Network RMSE: 112.65965444123857


In [18]:
# Initialize and train the model
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

# Predict and evaluate
y_pred = tree.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Decision Tree RMSE: {rmse}")

Decision Tree RMSE: 113.19691563852396


In [ ]:
#Would use the models that have an RMSE of 113 hours or less. 
#These numbers we would want to bring down further as possible issues are mitigated
#We are not taking in to account man power or the material used in the field.